# Import packages

In [ ]:
import pandas as pd
import geopandas as gpd

from helper_functions import open_wmata_station_data

# Notebook settings

In [2]:
FILES_TO_USE = ['Train_Movement_20190801_20191231.csv',
                'Train_Movement_20200101_20201231.csv']
WMATA_STATION_FILEPATH = r'Data\GIS_RAIL_STN_PNT.csv'
# Dates gotten from last codeblock of previous notebook, follows from GTFS data
BEGIN_DATE = '2019-09-14'
END_DATE = '2020-03-12'

# Open Data

In [3]:
wmata_stop_datetimes_dfs = []

for file in FILES_TO_USE:
    wmata_stop_datetimes_i = pd.read_csv(f"Data\\trainmovement\\{file}", usecols=['TRIP_ROUTE', 'BERTH_ID', 'NEXT_BERTH', 'ARRIVAL', 'TRIP_KEY'])
    wmata_stop_datetimes_dfs.append(wmata_stop_datetimes_i)

wmata_stop_datetimes = pd.concat(wmata_stop_datetimes_dfs)
print(f"{len(wmata_stop_datetimes):_}")
wmata_stop_datetimes.head(3)

13_562_714


,TRIP_ROUTE,BERTH_ID,NEXT_BERTH,ARRIVAL,TRIP_KEY
0,OR,D12-1,D13-1,01-AUG-2019 00:17:23,156055473
1,NaN,E02-1,E03-1,01-AUG-2019 00:10:15,156055542
2,NaN,E10-2,NaN,01-AUG-2019 00:09:15,156055571


In [4]:
wmata_stations = open_wmata_station_data(WMATA_STATION_FILEPATH)
wmata_stations['STATION_ID_LIST'] = [station_id.split('/') for station_id in wmata_stations['STATION_ID'].values]
print(len(wmata_stations))

97


# Filter to same date range as GTFS

In [5]:
# Convert arrival times to useable datetime objects
wmata_stop_datetimes['ARRIVAL'] = pd.to_datetime(wmata_stop_datetimes['ARRIVAL'], format='%d-%b-%Y %H:%M:%S')
wmata_stop_datetimes = wmata_stop_datetimes.sort_values(by=['ARRIVAL'])
wmata_stop_datetimes = wmata_stop_datetimes[wmata_stop_datetimes['ARRIVAL'] >= BEGIN_DATE]
wmata_stop_datetimes = wmata_stop_datetimes[wmata_stop_datetimes['ARRIVAL'] < END_DATE]
print(f"{len(wmata_stop_datetimes):_}")
wmata_stop_datetimes.head(3)

6_155_182


,TRIP_ROUTE,BERTH_ID,NEXT_BERTH,ARRIVAL,TRIP_KEY
1426905,BL,C10-1,C09-1,2019-09-14 00:00:00,157107609
1428274,RD,A02-1,A01-1,2019-09-14 00:00:02,157107847
1428060,NaN,A05-1,A04-1,2019-09-14 00:00:03,157107199


# Convert data to useable format

In [6]:
# Rename line names to full names for better readability
wmata_stop_datetimes['TRIP_ROUTE'] = wmata_stop_datetimes['TRIP_ROUTE'].replace({'RD': 'RED',
                                                                                 'SV': 'SILVER',
                                                                                 'OR': 'ORANGE',
                                                                                 'BL': 'BLUE',
                                                                                 'GR': 'GREEN',
                                                                                 'YL': 'YELLOW',
                                                                                 'BL|OR|SV': 'BLUE, ORANGE, SILVER',
                                                                                 'BL|SV': 'BLUE, SILVER',
                                                                                 'BL|YL': 'BLUE, YELLOW',
                                                                                 'GR|YL': 'GREEN, YELLOW',
                                                                                 'OR|SV': 'ORANGE, SILVER'})
# Convert berth_id's to station/stop ids by removing berth-specific affix
wmata_stop_datetimes['BERTH_ID'] = [value[:3] if pd.notna(value) else pd.NA for value in wmata_stop_datetimes['BERTH_ID'].values]
wmata_stop_datetimes['NEXT_BERTH'] = [value[:3] if pd.notna(value) else pd.NA for value in wmata_stop_datetimes['NEXT_BERTH'].values]

wmata_stop_datetimes.head(3)

,TRIP_ROUTE,BERTH_ID,NEXT_BERTH,ARRIVAL,TRIP_KEY
1426905,BLUE,C10,C09,2019-09-14 00:00:00,157107609
1428274,RED,A02,A01,2019-09-14 00:00:02,157107847
1428060,NaN,A05,A04,2019-09-14 00:00:03,157107199


# Correct stations names

In [7]:
# Some stations have double names (B01 and F01 are the same phyisical station for example), and so we create a correction dictionary
correction_dict = {}
for berth_id in wmata_stop_datetimes['BERTH_ID'].unique():
    if pd.notna(berth_id):
        matches = wmata_stations[[berth_id in station_list for station_list in wmata_stations['STATION_ID_LIST'].values]]
        if len(matches) > 0:
            correction_dict[berth_id] = matches['STATION_ID'].values[0]

In [8]:
# And then correct all stop names in the datetimes dataset
wmata_stop_datetimes['BERTH_ID'] = wmata_stop_datetimes['BERTH_ID'].replace(correction_dict)
wmata_stop_datetimes['NEXT_BERTH'] = wmata_stop_datetimes['NEXT_BERTH'].replace(correction_dict)
wmata_stop_datetimes = wmata_stop_datetimes.rename(columns={'BERTH_ID': 'STOP_ID', 'NEXT_BERTH': 'NEXT_STOP_ID'})
wmata_stop_datetimes.head(5)

,TRIP_ROUTE,STOP_ID,NEXT_STOP_ID,ARRIVAL,TRIP_KEY
1426905,BLUE,C10,C09,2019-09-14 00:00:00,157107609
1428274,RED,A02,A01/C01,2019-09-14 00:00:02,157107847
1428060,NaN,A05,A04,2019-09-14 00:00:03,157107199
1425092,GREEN,F05,F06,2019-09-14 00:00:05,157107893
1430064,RED,B07,B06/E06,2019-09-14 00:00:08,157107255


# Save file

In [9]:
wmata_stop_datetimes.to_csv(f"Intermediates\\wmata_stop_datetimes.csv", index=False)

### Open file

In [2]:
if wmata_dataset_year == 2019:
    path = f"Data\\trainmovement\\Train_Movement_{wmata_dataset_year}0801_{wmata_dataset_year}1231.csv"
else:
    path = f"Data\\trainmovement\\Train_Movement_{wmata_dataset_year}0101_{wmata_dataset_year}1231.csv"

wmata_stoptimes = pd.read_csv(path, usecols=['TRIP_ROUTE', 'BERTH_ID', 'NEXT_BERTH', 'ARRIVAL', 'TRIP_KEY'])
wmata_stoptimes.head(3)

,TRIP_ROUTE,BERTH_ID,NEXT_BERTH,ARRIVAL,TRIP_KEY
0,RD,B07-2,B06-2,01-JAN-2020 01:30:50,159662950
1,RD,B06-2,B05-2,01-JAN-2020 01:34:52,159662950
2,RD,B05-2,B04-2,01-JAN-2020 01:37:43,159662950


### Convert column values to desired formats

In [3]:
wmata_stoptimes['TRIP_ROUTE'] = wmata_stoptimes['TRIP_ROUTE'].replace({'RD': 'RED',
                                                                       'SV': 'SILVER',
                                                                       'OR': 'ORANGE',
                                                                       'BL': 'BLUE',
                                                                       'GR': 'GREEN',
                                                                       'YL': 'YELLOW',
                                                                       'BL|OR|SV': 'BLUE, ORANGE, SILVER',
                                                                       'BL|SV': 'BLUE, SILVER',
                                                                       'BL|YL': 'BLUE, YELLOW',
                                                                       'GR|YL': 'GREEN, YELLOW',
                                                                       'OR|SV': 'ORANGE, SILVER'})
wmata_stoptimes['BERTH_ID'] = [value[:3] if pd.notna(value) else pd.NA for value in wmata_stoptimes['BERTH_ID'].values]
wmata_stoptimes['NEXT_BERTH'] = [value[:3] if pd.notna(value) else pd.NA for value in wmata_stoptimes['NEXT_BERTH'].values]
wmata_stoptimes['ARRIVAL'] = pd.to_datetime(wmata_stoptimes['ARRIVAL'], format='%d-%b-%Y %H:%M:%S')
wmata_stoptimes = wmata_stoptimes.sort_values(by=['ARRIVAL'])
wmata_stoptimes

,TRIP_ROUTE,BERTH_ID,NEXT_BERTH,ARRIVAL,TRIP_KEY
1732,ORANGE,D04,D05,2020-01-01 00:00:00,159662834
1652,BLUE,C03,C04,2020-01-01 00:00:14,159661270
1639,ORANGE,K04,K05,2020-01-01 00:00:18,159661231
561,ORANGE,D11,D10,2020-01-01 00:00:18,159662886
491,BLUE,C12,C11,2020-01-01 00:00:23,159661803
...,...,...,...,...,...
8363436,YELLOW,F03,F02,2020-12-30 23:58:29,165573267
8364725,NaN,N06,N04,2020-12-30 23:59:07,165573003
8379535,YELLOW,F02,F01,2020-12-30 23:59:27,165573267
8378361,NaN,E10,<NA>,2020-12-30 23:59:30,165573158


In [ ]:
# WMATA stations
wmata_stations = pd.read_csv(r'Data\GIS_RAIL_STN_PNT.csv', sep=',', usecols=['NAME', 'X_COORD', 'Y_COORD', 'DISPLAY_NAME'])
wmata_stations = gpd.GeoDataFrame(wmata_stations, geometry=gpd.points_from_xy(wmata_stations['X_COORD'], wmata_stations['Y_COORD']))
wmata_stations['STATION_ID'] = [name.split('(')[-1][:-1] for name in wmata_stations['DISPLAY_NAME']]
wmata_stations = wmata_stations.drop(columns=['X_COORD', 'Y_COORD', 'DISPLAY_NAME'])
wmata_stations = wmata_stations.set_crs('EPSG:4326')
wmata_stations = wmata_stations.to_crs('EPSG:2248')
wmata_stations.head(3)

In [4]:
wmata_stations = pd.read_csv(r'Data\GIS_RAIL_STN_PNT.csv', sep=',', usecols=['NAME', 'X_COORD', 'Y_COORD', 'DISPLAY_NAME'])
wmata_stations = gpd.GeoDataFrame(wmata_stations, geometry=gpd.points_from_xy(wmata_stations['X_COORD'], wmata_stations['Y_COORD']))
wmata_stations['STATION_ID'] = [name.split('(')[-1][:-1] for name in wmata_stations['DISPLAY_NAME']]
wmata_stations['STATION_ID_LIST'] = [station_id.split('/') for station_id in wmata_stations['STATION_ID'].values]
wmata_stations = wmata_stations.drop(columns=['X_COORD', 'Y_COORD', 'DISPLAY_NAME'])
wmata_stations = wmata_stations.set_crs('EPSG:4326')
wmata_stations = wmata_stations.to_crs('EPSG:2248')
wmata_stations.head(5)

,NAME,geometry,STATION_ID,STATION_ID_LIST
0,SPRING HILL,POINT (1243521.877 459890.885),N04,[N04]
1,WIEHLE-RESTON EAST,POINT (1215548.021 466761.435),N06,[N06]
2,MCLEAN,POINT (1252439.294 458093.898),N01,[N01]
3,GREENSBORO,POINT (1245713.155 456992.08),N03,[N03]
4,U STREET/AFRICAN-AMER CIVIL WAR MEMORIAL/CARDOZO,POINT (1304503.807 455356.457),E03,[E03]


In [5]:
correction_dict = {}
for berth_id in wmata_stoptimes['BERTH_ID'].unique():
    if pd.notna(berth_id):
        matches = wmata_stations[[berth_id in station_list for station_list in wmata_stations['STATION_ID_LIST'].values]]
        
        if len(matches) > 0:
            correction_dict[berth_id] = matches['STATION_ID'].values[0]

In [6]:
wmata_stoptimes['BERTH_ID'] = wmata_stoptimes['BERTH_ID'].replace(correction_dict)
wmata_stoptimes['NEXT_BERTH'] = wmata_stoptimes['NEXT_BERTH'].replace(correction_dict)
wmata_stoptimes = wmata_stoptimes.rename(columns={'BERTH_ID': 'STOP_ID',
                                                  'NEXT_BERTH': 'NEXT_STOP_ID'})
wmata_stoptimes.head(40)

,TRIP_ROUTE,STOP_ID,NEXT_STOP_ID,ARRIVAL,TRIP_KEY
1732,ORANGE,D04,D05,2020-01-01 00:00:00,159662834
1652,BLUE,C03,C04,2020-01-01 00:00:14,159661270
1639,ORANGE,K04,K05,2020-01-01 00:00:18,159661231
561,ORANGE,D11,D10,2020-01-01 00:00:18,159662886
491,BLUE,C12,C11,2020-01-01 00:00:23,159661803
2136,GREEN,D03/F03,F04,2020-01-01 00:00:23,159661964
760,SILVER,C05,C04,2020-01-01 00:00:28,159662147
2058,BLUE,A01/C01,D01,2020-01-01 00:00:32,159661914
2112,ORANGE,K02,K01,2020-01-01 00:00:34,159662157
1081,ORANGE,D02,D01,2020-01-01 00:00:34,159662571


### Save file

In [7]:
wmata_stoptimes.to_csv(f"Intermediate Results\\WMATA_stoptimes_{wmata_dataset_year}.csv", index=False)